# Abordagem Inicial 

## Objetivo 

### Propor método para:
* encontrar representações comuns de diferentes modalidades sensoriais
* aumentar a quantidade de dados utilizados para redução de dimensionalidade do sinal
* melhorar a sensibilidade de análises MVPA posteriores 

### Inicialmente codificar cada instante de tempo (sem usar janelas no tempo)
### Decodificar o momento que estava na mesma e na outra modalidade a partir do estado reduzido
* Comparar com decodificação usando: (1) Sinais originais, (2) PCA geral, (3) PCA em cada ponto, (4) Auto-encoder



### Importações necessárias para execução desse notebook

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns; sns.set()
import sys

from os import listdir
from os.path import isfile, join

from sklearn.decomposition import PCA

sys.path.append("../src/data")
from myMNE import makeMNE

### Carregando dados auditivos e visuais

In [2]:
PATH_AUD = '../data/raw/aud'
PATH_VIS = '../data/raw/vis'

PATH_INFO = '../data/raw/info_'


### Funções auxiliares para leitura do arquivo

In [3]:
def files_in_path(path):
    return [path+"/"+f for f in listdir(path) if isfile(join(path, f))]

# Getting data from MNE strutuct


def data_from_mne(files):
    return np.array([file.get_data().T for file in files])

In [4]:
# List of Files contained in the folder with path

path_file_aud = files_in_path(PATH_AUD)
path_file_vis = files_in_path(PATH_VIS)

In [ ]:
# Reading files with the MNE library

files_aud = list(map(makeMNE, path_file_aud))
files_vis = list(map(makeMNE, path_file_vis))

In [ ]:
# Getting data in numpy format

data_aud = data_from_mne(files_aud)
data_vis = data_from_mne(files_vis)

### Checking the shape in arrays

In [ ]:
print("Data Aud with shape: {} (individual, raw, channels, trial)".format(data_aud.shape))
print("Data Vis with shape: {} (individual, raw, channels, trial)".format(data_vis.shape))

In [ ]:
# Function to exposure separation:
# by definition in trial odd are the first exposure, and trial even the second exposure

def _exposure_1(data): 
    # [individual, raw, channels, trial]
    return data[::, ::, ::, 0:: 2]

def _exposure_2(data): 
    # [individual, raw, channels, trial]
    return data[::, ::, ::, 1:: 2]

In [ ]:
## Splitting the exposures for all the individuals

aud_expo_1 = _exposure_1(data_aud)

aud_expo_2 = _exposure_2(data_aud) 

vis_expo_1 = _exposure_1(data_vis)

vis_expo_2 = _exposure_2(data_vis)

In [ ]:
# Auxiliary function to calculate the average in the bin


def _average(frame):
    return np.average(frame, axis=0)


# Function for splitting and compute the average in the bins
def six_bin_first(exposure):
    """Splitting array in bins for the first exposure,
    and compute the average

    Parameters
    ----------
    exposure: numpy.array (3-dimensions) [raw, channels, trial]
       Array containing data from an exposure.
    Returns
    -------
    average_frames: list of numpy.array
        average of 6 arrays with 31 points each
    """
    # Indices

    # ---------------------------------------------------------------------

    # If the zero is not at the starting point just change the value.
    zero = 0

    # End in 1500 ms
    end = 1500//4 + zero

    # 6 bins with 31 points from 250 Hz

    size = 744 // 4

    # ---------------------------------------------------------------------
    time_frame = exposure[end-size:end]

    frames = np.split(time_frame, 6)

    average_frames = np.array(list(map(_average, frames)))

    return average_frames

In [ ]:
# Function for splitting and compute the average in the bins, for the second bins
def six_bin_second(exposure, indice_exposure):
    """Splitting array in bins for the first exposure,
    and compute the average

    Parameters
    ----------
    exposure: numpy.array (3-dimensions) [raw, channels, trial]
       Array containing data from an exposure.
    Returns
    -------
    average_frames: list of numpy.array
        Average of 6 arrays with 31 points each
    """
    # Indices

    # ---------------------------------------------------------------------

    # Indice moment of exposure, subtracting 124 milliseconds (31 points)
    #import pdb; pdb.set_trace()
    begin = indice_exposure - 31

    # 6 bins with 31 points from 250 Hz

    size = 744 // 4

    # ---------------------------------------------------------------------
    time_frame = exposure[begin:begin+size]

    frames = np.split(time_frame, 6)

    average_frames = np.array(list(map(_average, frames)))

    return average_frames

In [ ]:
import scipy.io as sio


def indice_time_delay(name: str):

    info_path = PATH_INFO+name

    infos = list(map(sio.loadmat, files_in_path(info_path)))

    time_delay = infos[0]['report']['all_trials_delay'][0][0][0]

    return ((np.around((time_delay*250), decimals=0)))

In [ ]:
indice_vis_expo = indice_time_delay('vis').astype('int')
indice_aud_expo = indice_time_delay('aud').astype('int')

### Average in the bins
----

In [ ]:
aud_expo_1_frames = list(map(six_bin_first, aud_expo_1))
vis_expo_1_frames = list(map(six_bin_first, vis_expo_1))

In [ ]:
aud_expo_2_frames = list(map(six_bin_second, aud_expo_2, indice_vis_expo))
vis_expo_2_frames = list(map(six_bin_second, vis_expo_2, indice_aud_expo))

----

## Com isso, iremos para o próximo passo, que consiste de aplicar o PCA para reduzir as dimensões redudantes presentes nos múltiplos canais.

Não realizamos nesse momento a separação do conjunto treino-teste, seguindo o autor original.


![aquisicao](https://raw.githubusercontent.com/bruAristimunha/Across-Modalities/master/reports/figures/mapa_rapha.jpg)
#### Figura 01: Diagrama de Reunião de Orientação. O ponto de interesse está na tabela para aplicação do PCA. Segundo os autores, eles empregaram em média 32 componentes +- 1


------------------------------------

# Studying of the number of components

In [ ]:
def myPCA(data: np.array, return_data=False, n_components=0.99):

    pca_ = PCA(random_state=42, svd_solver='full', n_components=n_components)
    
    data = data.T

    data_ft = pca_.fit_transform(data)

    if(return_data):
        return data_ft
    else:
        return pca_

In [ ]:
def myPCA_method(data): 
    return myPCA(data, return_data=False)
def myPCA_data(data): 
    return myPCA(data, return_data=True)

In [ ]:
pcas = []

for person in aud_expo_1_frames:
    method_pca = list(map(myPCA_method,person))
    pcas.append(method_pca)
    
for person in vis_expo_1_frames:
    method_pca = list(map(myPCA_method,person))
    pcas.append(method_pca)
    
for person in aud_expo_2_frames:
    method_pca = list(map(myPCA_method,person))
    pcas.append(method_pca)
    
for person in vis_expo_2_frames:
    method_pca = list(map(myPCA_method,person))
    pcas.append(method_pca)

-------

In [ ]:
import matplotlib.pylab as plt
import matplotlib

font = {'weight' : 'bold',
        'size'   : 18}

matplotlib.rc('font', **font)

In [ ]:
fig, ax = plt.subplots(nrows=2, figsize=(10, 25))
method_pca = np.array(pcas).reshape(-1)
componentes = [pc.n_components_ for pc in method_pca]

for pc in method_pca:

    ax[0].plot(np.arange(1, pc.n_components_ + 1),
             pc.explained_variance_ratio_, '+', linewidth=2)
    ax[0].set_ylabel('Explicação da variação do PCA')

ax[1] = sns.boxplot(componentes,orient='vert',ax=ax[1])
ax[1].set_title("média {media:.5}, desvio padrao {std:.2}, maximo {maxi}, minimo {mini}".format(
                                                                        media = np.mean(componentes),
                                                                        std   = np.std(componentes), 
                                                                        maxi  = np.max(componentes),
                                                                        mini  = np.min(componentes)))
plt.show()

## Escolhemos $47$ como o número de componentes, sendo esse o número máximo de componentes necessários para explicar 99% dos dados.

In [ ]:
def myPCA_47_comp(data, n_components = 47): 
    return myPCA(data, return_data=True, n_components=n_components)

In [ ]:
#####################################################################    

X_aud_expo_1_frames = []

for person in aud_expo_1_frames:

    data_pca = list(map(myPCA_47_comp, person))
    X_aud_expo_1_frames.append(data_pca)
#####################################################################    

X_aud_expo_2_frames = []


for person in aud_expo_2_frames:

    data_pca = list(map(myPCA_47_comp, person))
    X_aud_expo_2_frames.append(data_pca)
    
#####################################################################

X_vis_expo_1_frames = []

for person in vis_expo_1_frames:

    data_pca = list(map(myPCA_47_comp, person))
    X_vis_expo_1_frames.append(data_pca)
    
#####################################################################    
        
X_vis_expo_2_frames = []

for person in vis_expo_2_frames:

    data_pca = list(map(myPCA_47_comp, person))
    X_vis_expo_2_frames.append(data_pca)
    
#####################################################################        
    
    

In [ ]:
def to_DataFrame(X :list)-> pd.DataFrame:
    
    X_transpose = np.array(X).transpose([1, 2, 0, 3])
    
    X_preDataFrame = np.concatenate(np.concatenate(X_transpose,axis=0),axis=0)
    
    dataframe = pd.DataFrame(X_preDataFrame)
    
    dataframe = dataframe.reset_index()
    
    dataframe['person'] = (dataframe['index'] % 20)+1
    
    dataframe['trial'] = (dataframe['index'] % 120)+1
    
    dataframe['class'] = (dataframe['index'] % 6) +1
    
    dataframe = dataframe.drop('index',1)
    
    return dataframe

In [ ]:
df_vis_1 = to_DataFrame(X_vis_expo_1_frames)
df_vis_2 = to_DataFrame(X_vis_expo_2_frames)

df_aud_1 = to_DataFrame(X_aud_expo_1_frames)
df_aud_2 = to_DataFrame(X_aud_expo_2_frames)

In [ ]:
df_vis_1.to_csv("../data/processed/visual_exposure_1.csv",index=None)
df_vis_2.to_csv("../data/processed/visual_exposure_2.csv",index=None)

df_aud_1.to_csv("../data/processed/auditory_exposure_1.csv",index=None)
df_aud_2.to_csv("../data/processed/auditory_exposure_2.csv",index=None)


In [ ]:
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

def make_Classification(df):
    X = df.drop(['person',  'trial', 'class'],1)
    y = df['class']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

    clf = GaussianNB()
    clf = clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    print(classification_report(y_test, y_pred))

In [ ]:
print("Classificação nos Dados Visual, exposição 1")
make_Classification(df_vis_1)

In [ ]:
print("Classificação nos Dados Visual, exposição 2")
make_Classification(df_vis_2)

In [ ]:
print("Classificação nos Dados Auditivo, exposição 1")
make_Classification(df_aud_1)

In [ ]:
print("Classificação nos Dados Auditivo, exposição 2")
make_Classification(df_aud_2)

### EEG recordings and pre-processing

EEG was recorded continuously from 64 ActiCap Electrodes (Brain Products) at 1000 Hz by a QuickAmp amplifier (Brain Products). All sites were referenced to FCz and grounded to AFz. The electrodes were positioned according to the International 10–10 system. Additional bipolar electrodes registered the electrooculogram (EOG).



EEG pre-processing was carried out using BrainVision Analyzer (Brain Products). All data were down-sampled to 250 Hz and re-referenced to the average activity across electrodes. For eye movement artefact rejection, an independent component analysis (ICA) was performed on filtered (Butterworth Zero Phase Filter between 0.05.05 Hz and 30 Hz) and segmented (−200 to 2000 ms relative to S1) data. For the ICA, epochs were baselined based on the average activity of the entire trial. Eye related components were identified by comparing individual ICA components with EOG channels and by visual inspection. The average proportion of rejected trials for each participant corresponded to 0.0613. For all further analyses, epochs were baseline corrected based on the period between −200 ms and 0 ms relative to S1 presentation.

### Similarity between different trials and modalities

To calculate the similarity across trials we used a bootstrap approach. For each participant and comparison of interest, data from all intervals that lasted at least 1.125 s were divided into two groups of trials and averaged across trials per group (Bueno et al., 2017). Then, for each time point, data from each split (two-row vectors with the averaged amplitude values of all 62 electrodes) were compared using a Pearson correlation. This procedure was repeated 5000 times for each participant and Fisher transformed coefficients were averaged across permutations for each participant and comparison of interest.

At the group level, EEG-analyses were implemented non-parametrically (Maris and Oostenveld, 2007, Wolff et al., 2015) with sign-permutation tests. For each time-point, the Fisher transformed coefficients for a random half of the participants were multiplied by. The resulting distribution was used to calculate the p-value of the null-hypothesis that the mean value was equal to 0. Cluster-based permutation tests were then used to correct for multiple comparisons across time using 5000 permutations, with a cluster-forming threshold of p (Maris and Oostenveld, 2007). The sum of the values within a cluster was used as the cluster-level statistic. The significance threshold was set at p; all tests were one-sided.

### Decoding the elapsed interval

To investigate whether information about the elapsed interval could be decoded from activity across electrodes, we used a Naive Bayes classifier to perform a multiclass classification (Grootswagers et al., 2017). For all classifications, activity from the last 125 ms of the elapsed interval (−125 ms to 0 relative to S2) was averaged for each electrode. Data for each exposure (E1 and E2) were analysed separately. Intervals were divided into six equally sized bins (125 ms each).